<h1>Logistical Regression</h1>

In [1]:
from IPython.display import HTML
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import metrics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymysql
plt.style.use('Solarize_Light2')
%matplotlib inline
# Increase default figure and font sizes for easier viewing.
plt.rcParams['figure.figsize'] = (16, 9)
plt.rcParams['font.size'] = 14

In [2]:
%%time
host = 'sydney-fp-test.clgfypx3smmh.ap-southeast-2.rds.amazonaws.com'
database = 'fitnessplayground'
user = 'fpdb'
passw = 'fsmbfp2017'
conn = pymysql.connect(host = host, db=database, user=user, password=passw)
membership_query = """
SELECT enrol.createDate, enrol.gymName, enrol.staffName, enrol.membershipType, enrol.trainingStarterPack, 
enrol.noMinimumTerm, enrol.couponButton, enrol.gender, enrol.dob, enrol.zip, cxl.createDate, cxl.nps, cxl.staffName, cxl.cxlReason, 
DATEDIFF(cxl.createDate, enrol.createDate) as Tenure 
FROM EnrolmentSubmissionData enrol LEFT JOIN CancellationSubmissionData cxl 
ON enrol.email = cxl.email AND enrol.phone = cxl.phone 
WHERE DATEDIFF(cxl.createDate, enrol.createDate) > 0 
OR DATEDIFF(cxl.createDate, enrol.createDate) IS NULL
"""

# tenure_query = """
# SELECT enrol.dob, enrol.ongoingFnDirectDebitTotal, DATEDIFF(cxl.createDate, enrol.createDate) as tenure, cxl.nps, enrol.zip 
# FROM EnrolmentSubmissionData enrol 
# INNER JOIN CancellationSubmissionData cxl ON enrol.phone = cxl.phone AND enrol.email = cxl.email 
# WHERE DATEDIFF(cxl.createDate, enrol.createDate) > 0
# AND cxl.nps IS NOT NULL
# AND enrol.ongoingFnDirectDebitTotal IS NOT NULL
# AND enrol.dob IS NOT NULL
# ORDER BY 2 DESC;
# """

# conn_str = "host='{}' db='{}' user='{}' password='{}'".format(host, database, user, passw)
# conn = pymysql.connect(conn_str)
conn = pymysql.connect(host = host, db=database, user=user, password=passw)
membership = pd.read_sql(membership_query, con=conn)
membership.columns = ['Start_Date','Gym_Location','Enrol_Staff_Name','Membership_Type','Training_Starter_Pack','Twelve_Month_Contract','Coupon','Gender','DOB','Postcode','Cancel_Date','NPS','Cancel_Staff_Name','Cancellation_Reason','Tenure']
# tenure = pd.read_sql(tenure_query, con=conn)
conn.close()

CPU times: user 609 ms, sys: 66.9 ms, total: 676 ms
Wall time: 16.5 s


In [3]:
membership['DOB'] = pd.to_datetime(membership['DOB'])
membership['Age'] = datetime.now().year - membership['DOB'].dt.year

In [4]:
membership.shape

(8406, 16)

In [5]:
membership.dtypes

Start_Date               datetime64[ns]
Gym_Location                     object
Enrol_Staff_Name                 object
Membership_Type                  object
Training_Starter_Pack            object
Twelve_Month_Contract            object
Coupon                           object
Gender                           object
DOB                      datetime64[ns]
Postcode                         object
Cancel_Date              datetime64[ns]
NPS                              object
Cancel_Staff_Name                object
Cancellation_Reason              object
Tenure                          float64
Age                             float64
dtype: object

In [6]:
membership['member_has_cancelled'] = np.where(membership.Cancel_Date.isnull(), 0, 1)
# membership['has_cancelled']

<h2>Set Target variable. Has the member cancelled their Membership?</h2>
<h5>1 = True, the member has cancelled their memebership</h5>
<h5>0 = False, the member has NOT cancelled their memebership</h5>

In [7]:
membership.member_has_cancelled.value_counts()

0    6376
1    2030
Name: member_has_cancelled, dtype: int64

<h2>Clean the Data</h2>

In [8]:
# membership.Training_Starter_Pack.drop()
np.where(membership.Training_Starter_Pack.isnull())

(array([7988, 7989]),)

In [9]:
# axis=0 for rows, 1 for columns
# membership.Training_Starter_Pack.drop(membership.index[291], axis=0, inplace=True)
# membership.Training_Starter_Pack.drop(membership.index[293], axis=0, inplace=True)
# membership.drop(membership.index[291], axis=0, inplace=True)
# membership.drop(membership.index[292], axis=0, inplace=True)

In [10]:
membership.Training_Starter_Pack.isnull().sum()

2

In [11]:
# membership.Training_Starter_Pack.head(294)

In [12]:
membership.Membership_Type = membership.Membership_Type.map(
{
    'NON-MEMBER':0,
    'GYM':1,
    'FIT':2,
    'PLAY':3,
    'BUNKER GYM':4,
    'BUNKER PLAY':5
})

membership.Gym_Location = membership.Gym_Location.map(
{
    'Bunker':0,
    'Gateway':1,
    'Marrickville':2,
    'Newtown':3,
    'Surry Hills':4
})

membership.Training_Starter_Pack = membership.Training_Starter_Pack.map(
    {'I\'m a Fitness Guru':0, 
     '0':0, 
     'One On One':1,
     'Personalise Group Training':2,
     'Transformer':3,
     'Ongoing Personal Training':4
    })

membership.Gender = membership.Gender.map(
{
    'Female':0,
    'Male':1
})

membership['Staff_Key'] = membership.Enrol_Staff_Name.map(
{
    'Stephanie Thomas':0,
    'Giannino Terilli':1,
    'Luke Wood':2,
    'Trung Bui':3,
    'George Wall':4,
    'Isabella Murphy':5,
    'Jodie Fenwick':6,
    'Jessica Saffiotti':7,
    'Tim Air':8,
    'Simon Lee':9,
    'Jess Brewster':10,
    'Toree Read':11,
    'Kris Andre':12,
    'Laura Schellenberger':13,
    'Emily Wilson':14,
    'Acalya Cingoz':15,
    'Leanne Forresst':16,
    'Lara McKee':17,
    'Amy Jamieson':18,
    'Jeremy Johnson':19,
    'Georgia Shimmin':20,
    'Daniel Bowden':21,
    'Quintin Venter':22,
    'Leanne Forrest':23,
    'Tash Zetting':24,
    'Dan Bowden':25,
    'Carolina Araya':26,
    'Leanne Hunt':27,
    'Frank Pop':28,
    'Dan Dowden':29,
    'Hannah Ghaus':30,
    'James Garland':31,
    'Katie Hayes':32,
    'Sam Okojie':33,
    'Anthony Novak':34,
    'Noren Woo':35,
    'Kate Brush':36,
    'Francesco Nisi':37
})

membership.Twelve_Month_Contract.replace('10 | Yes + $5.00 per week',1, inplace=True)
membership.Twelve_Month_Contract.replace('10.00 | Yes + $5.00 per week',1, inplace=True)
membership.Twelve_Month_Contract.replace('Yes + $5.00 per week',1, inplace=True)
membership.Twelve_Month_Contract.replace('$10.00 | Yes + $5.00 per week',1, inplace=True)
membership.Twelve_Month_Contract.replace('0 | No Thank You',0, inplace=True)
membership.Twelve_Month_Contract.replace('No Thank You',0, inplace=True)
membership.Twelve_Month_Contract.fillna(0, inplace=True)
membership.Coupon.replace('Apply Coupon?',1, inplace=True)
membership.Coupon.replace('Apply Coupon',1, inplace=True)
membership.Coupon.replace('Cancel',0, inplace=True)
membership.fillna(0, inplace=True)

<h2>Double Check the Data</h2>

In [13]:
# membership.Training_Starter_Pack.value_counts()
# membership.Staff_Key.isnull().sum()
# membership.Enrol_Staff_Name.isnull().sum()
# membership.Enrol_Staff_Name.value_counts()

In [14]:
membership.columns

Index(['Start_Date', 'Gym_Location', 'Enrol_Staff_Name', 'Membership_Type',
       'Training_Starter_Pack', 'Twelve_Month_Contract', 'Coupon', 'Gender',
       'DOB', 'Postcode', 'Cancel_Date', 'NPS', 'Cancel_Staff_Name',
       'Cancellation_Reason', 'Tenure', 'Age', 'member_has_cancelled',
       'Staff_Key'],
      dtype='object')

<h2>Check the Data</h2>

In [21]:
features = ['Staff_Key','Membership_Type','Gym_Location','Twelve_Month_Contract','Coupon','Gender','Age']#,'member_has_cancelled']

In [16]:
membership_a = membership[features]
for feat in features:
    print(membership_a[feat].value_counts())

0.0     829
1.0     749
2.0     720
3.0     710
4.0     693
5.0     599
6.0     514
7.0     382
8.0     363
9.0     315
11.0    283
10.0    281
12.0    277
13.0    259
14.0    190
15.0    164
21.0    150
16.0    146
18.0    138
17.0    135
20.0    112
19.0    108
22.0    101
23.0     59
24.0     50
26.0     20
28.0     18
27.0     15
30.0     11
31.0      5
32.0      3
34.0      2
33.0      2
35.0      1
36.0      1
37.0      1
Name: Staff_Key, dtype: int64
1    4285
3    3001
2     707
4     235
5     177
0       1
Name: Membership_Type, dtype: int64
3    2070
4    2030
1    1965
2    1928
0     413
Name: Gym_Location, dtype: int64
0.0    6241
1.0    2165
Name: Twelve_Month_Contract, dtype: int64
1.0    5362
0.0    3044
Name: Coupon, dtype: int64
0    4290
1    4116
Name: Gender, dtype: int64
26.0    605
28.0    565
27.0    550
25.0    538
24.0    488
29.0    472
31.0    416
30.0    409
23.0    406
32.0    319
33.0    304
22.0    301
21.0    249
34.0    242
35.0    204
36.0    198
20.

<h2>Build the Model</h2>

**Set Feature Variables. y = member_has_cancelled.**

In [18]:
# membership_a[features]
# memberships_scaled

In [23]:
# X and y features
X1 = membership_a[features]#.drop('member_has_cancelled', axis =1)
y1 = membership_a['member_has_cancelled']

# Implement this and see if it improves the model
scaler = StandardScaler()
memberships_scaled = pd.DataFrame(scaler.fit_transform(X1)) 
memberships_scaled.columns = features
# X_test = scaler.transform(X_test)

membership_md1 = pd.get_dummies(memberships_scaled)#, drop_first = True) # this drops the first column
# weight = balanced made a huge difference
LogReg1 = LogisticRegression(class_weight='balanced')

# using train test split to cross val
x_train1, x_test1, y_train1, y_test1 = train_test_split(memberships_scaled, y1, random_state =423)

# fit model
LogReg1.fit(x_train1, y_train1)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [ ]:
membership_md1.head()

**Get the Coefficient for each feature.**

*Seems like `Twelve_Month_Contract, Coupon, Gender` carry that largest coef.*

In [26]:
# name = membership_md1.columns.drop('member_has_cancelled')
name = memberships_scaled
print(LogReg1.coef_)
coef = abs(LogReg1.coef_[0])

pd.DataFrame([name,coef],index = ['Name','Coef']).transpose().sort_values(by='Coef', ascending=False)

[[ 0.00225596 -0.05186791  0.11605001  0.27954411 -0.24971725 -0.14796981
  -0.13596793]]


,Name,Coef
0,Staff_Key Membership_Type Gym_Location...,"[0.0022559634624640533, 0.05186791048262737, 0..."


In [27]:
y_pred = LogReg1.predict(x_test1)
metrics.accuracy_score(y_test1,y_pred)

0.5913415794481446

<img src="https://www.dataschool.io/content/images/2015/01/confusion_matrix2.png" alt="Confusion Matrix" height="500" width="500">



In [28]:
metrics.confusion_matrix(y_test1,y_pred)

array([[933, 668],
       [191, 310]])

**Receiver operating characteristic**
<p>https://en.wikipedia.org/wiki/Receiver_operating_characteristic</p>
<p>.90-1 = excellent (A)</p>
<p>.80-.90 = good (B)</p>
<p>.70-.80 = fair (C)</p>
<p>.60-.70 = poor (D)</p>
<p>.50-.60 = fail (F)</p>

In [29]:
# Before balancing 0.5051793
# weighting ups to 0.660
metrics.roc_auc_score(y_test1,y_pred)

0.6007616247829138

*Lower is better*

In [ ]:
metrics.log_loss(y_test1,y_pred)

<h2>K Folds</h2>

In [30]:
for folds in range(3,7):
    print('*____*____*____*____*____*____*____*____*')
    print('k: {}'.format(folds))
    model = LogisticRegression(class_weight='balanced')
    scores = cross_val_score(model, X1, y1, cv=folds)
    predictions = cross_val_predict(model, X1, y1, cv=folds)
    accuracy = metrics.accuracy_score(y1,predictions)
    confusion_matrix = metrics.confusion_matrix(y1,predictions)
    log_loss = metrics.log_loss(y1,predictions)
    print("Cross-Predicted Accuracy: {}".format(accuracy))
    print("Cross-Predicted Log Loss: {}".format(log_loss))
    print("Cross-Predicted Confusion Matrix:\n {}".format(confusion_matrix))
print('-____-____-____-____-____-____-____-____-')

*____*____*____*____*____*____*____*____*
k: 3
Cross-Predicted Accuracy: 0.6031406138472519
Cross-Predicted Log Loss: 13.707253906492777
Cross-Predicted Confusion Matrix:
 [[4102 2274]
 [1062  968]]
*____*____*____*____*____*____*____*____*
k: 4
Cross-Predicted Accuracy: 0.5902926481084939
Cross-Predicted Log Loss: 14.151031083318571
Cross-Predicted Confusion Matrix:
 [[3848 2528]
 [ 916 1114]]
*____*____*____*____*____*____*____*____*
k: 5
Cross-Predicted Accuracy: 0.5979062574351653
Cross-Predicted Log Loss: 13.88806091125959
Cross-Predicted Confusion Matrix:
 [[3905 2471]
 [ 909 1121]]
*____*____*____*____*____*____*____*____*
k: 6
Cross-Predicted Accuracy: 0.5869616940280752
Cross-Predicted Log Loss: 14.266083963939646
Cross-Predicted Confusion Matrix:
 [[3787 2589]
 [ 883 1147]]
-____-____-____-____-____-____-____-____-


**Try improve the model**

In [ ]:
y_pred_prob = LogReg1.predict_proba(x_test1)
y_pred_prob

Lets try one more thing before we revert to grabbing more features.  Adjusting the probability threshold.

Use the `LogisticRegression.predict_proba()` attribute to get the probabilities.

The first probability is the for class 0 and the second is for class 1

In [ ]:
y_pred_prob_t = y_pred_prob.transpose()
plt.hist(y_pred_prob_t[0])
plt.show()
plt.hist(y_pred_prob_t[1])

** Calculate a new threshold and use it to convert predicted probabilities to output classes**

Lets try decreaseing the threshold to %20 predicted probability or higher.

In [ ]:
y_pred2=[]
for prob in y_pred_prob_t[1]:
    if prob > .20:
        y_pred2.append(1)
    else:
        y_pred2.append(0)
        
print(len(y_pred2))
print(len(y_test1))

In [ ]:
y_pred2.count(1)  #Actually made some predictions

In [ ]:
metrics.accuracy_score(y_test1,y_pred2)

In [ ]:
metrics.confusion_matrix(y_test1,y_pred2)

In [ ]:
metrics.roc_auc_score(y_test1,y_pred2)

In [ ]:
metrics.log_loss(y_test1,y_pred2)

<h2>Regularization to optimise model.</h2>

In [32]:
# # X and y features
# X1 = membership_md1.drop('member_has_cancelled', axis =1)
# y1 = membership_md1['member_has_cancelled']

X1 = membership_a[features]#.drop('member_has_cancelled', axis =1)
y1 = membership_a['member_has_cancelled']

# using train test split to cross val
x_train, x_test, y_train, y_test = train_test_split(X1,y1, random_state =42)

cees = [0.01, 0.1, 1.0, 10, 100]
# cees = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7 ,1.8, 1.9]

for c in cees:
    logreg = LogisticRegression(class_weight='balanced', penalty='l2', C=c)
    logreg.fit(x_train,y_train)
    y_pred = logreg.predict(x_test)
    roc = metrics.roc_auc_score(y_test, y_pred)
    print(roc," : ", c)

0.5787031277537606  :  0.01
0.5821346375143843  :  0.1
0.5822681139136836  :  1.0
0.5830158409272334  :  10
0.5833229662343586  :  100


<h2>Conclusion: The model is not accurate enough to be a relaible predictor of a member cancelling with Logistical Regression.

Trying with Random Forrest

</h2>

In [34]:
##
## Random Forest
##
acc = []
log = []
i = 0
for est in range(50, 100, 20):
    for depth in range(2,10):
        rf = RandomForestClassifier(max_depth=depth, n_estimators=est,class_weight='balanced')
        rf.fit(x_train, y_train)
        scr = rf.score(x_test, y_test)
        details = 'Model {}, n_estimators={}, max-depth={} The mean accuracy={}'.format( i, est, depth, np.round(scr, 3) )
        print(details)
        acc.append(scr)
        log.append(details)
        # iterate
        i += 1

# Best Model
log[np.argmax(acc)]

Model 0, n_estimators=50, max-depth=2 The mean accuracy=0.596
Model 1, n_estimators=50, max-depth=3 The mean accuracy=0.606
Model 2, n_estimators=50, max-depth=4 The mean accuracy=0.634
Model 3, n_estimators=50, max-depth=5 The mean accuracy=0.637
Model 4, n_estimators=50, max-depth=6 The mean accuracy=0.648
Model 5, n_estimators=50, max-depth=7 The mean accuracy=0.666
Model 6, n_estimators=50, max-depth=8 The mean accuracy=0.675
Model 7, n_estimators=50, max-depth=9 The mean accuracy=0.679
Model 8, n_estimators=70, max-depth=2 The mean accuracy=0.606
Model 9, n_estimators=70, max-depth=3 The mean accuracy=0.616
Model 10, n_estimators=70, max-depth=4 The mean accuracy=0.623
Model 11, n_estimators=70, max-depth=5 The mean accuracy=0.629
Model 12, n_estimators=70, max-depth=6 The mean accuracy=0.641
Model 13, n_estimators=70, max-depth=7 The mean accuracy=0.655
Model 14, n_estimators=70, max-depth=8 The mean accuracy=0.673
Model 15, n_estimators=70, max-depth=9 The mean accuracy=0.679
Mo

'Model 15, n_estimators=70, max-depth=9 The mean accuracy=0.679'